In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
############Leemos el CSV######################
df = pd.read_csv('poker-hand-tiny.csv')

############Limpiamos el CSV###################
for i in range(5,10):
    #Creamos lista con los indices a eliminar
    idmemor = df[df['Clase']== i].index
    #Eliminamos los indices del csv
    df = df.drop(idmemor)

#Define cada dataset
y_train = df.iloc[:,10]
x_train = df.iloc[:,0:10]

In [3]:
###################### CREACIÓN DATOS SINTÉTICOS #####################
smote = SMOTE()
X_train, Y_train = smote.fit_sample(x_train, y_train)
print(X_train, Y_train)

        S1  C1  S2  C2  S3  C3  S4  C4  S5  C5
0        1   1   1  13   2   4   2   3   1  12
1        3  12   3   2   3  11   4   5   2   5
2        1   9   4   6   1   4   3   2   3   9
3        1   4   3  13   2  13   2   1   3   6
4        3  10   2   7   1   2   2  11   4   9
...     ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
751440   2   1   4  11   1  10   1  11   3  13
751441   1   1   1   5   3   2   2   4   1   4
751442   4   1   3  12   1  10   1  10   3  11
751443   4   8   4   7   4   7   2   4   1   5
751444   3   9   1   8   3   7   3   9   3  11

[751445 rows x 10 columns] 0         0
1         1
2         1
3         1
4         0
         ..
751440    4
751441    4
751442    4
751443    4
751444    4
Name: Clase, Length: 751445, dtype: int64


In [4]:
############# ÁRBOL DE DECISIÓN #################
arboldecision = DecisionTreeClassifier()
parametros = {'criterion': ['gini', 'entropy'],'splitter':['best', 'random'],'max_depth': [1,4,7,8,9,10]}

In [5]:
############# VALIDACIÓN CRUZADA - Grid Search ##############
clf = GridSearchCV(arboldecision, parametros, cv = 3)
clf.fit(X_train, Y_train)
clf.cv_results_
df = pd.DataFrame(clf.cv_results_)
print(df[['param_criterion', 'param_max_depth', 'param_splitter', 'mean_test_score', 'rank_test_score' ]])

   param_criterion param_max_depth param_splitter  mean_test_score  \
0             gini               1           best         0.225575   
1             gini               1         random         0.225250   
2             gini               4           best         0.264441   
3             gini               4         random         0.270068   
4             gini               7           best         0.318030   
5             gini               7         random         0.309243   
6             gini               8           best         0.347473   
7             gini               8         random         0.339141   
8             gini               9           best         0.376835   
9             gini               9         random         0.374228   
10            gini              10           best         0.409097   
11            gini              10         random         0.365663   
12         entropy               1           best         0.225575   
13         entropy  

In [6]:
######## NAIVE BAYES ##########
bayes = MultinomialNB()
val_cruzada = cross_validate(bayes, X_train, Y_train, cv = 3)
print(val_cruzada.keys())
print('Resulatado Naive Bayes: ', val_cruzada['test_score']*100)

dict_keys(['fit_time', 'score_time', 'test_score'])
Resulatado Naive Bayes:  [22.83038302 22.64993093 22.5566011 ]


In [7]:
model = clf.best_estimator_
print('Resultado Arbol de Decisión:' , model.score(X_train,Y_train)*100)


Resultado Arbol de Decisión: 41.05663089114972


In [23]:



############ Cuento componentes del árbol ###############

n_nodes = model.tree_.node_count
children_left = model.tree_.children_left
children_right = model.tree_.children_right
feature = model.tree_.feature
threshold = model.tree_.threshold

node_indicator = model.decision_path(X_train)

leave_id = model.apply(X_train)

sample_id = 0
node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                    node_indicator.indptr[sample_id + 1]]

print('Rules used to predict sample %s: ' % sample_id)
for node_id in node_index:
    if leave_id[sample_id] == node_id:
        continue

    if (X_test[sample_id, feature[node_id]] <= threshold[node_id]):
        print(X_test[sample_id, feature[node_id]])
        
        threshold_sign = "<="
    else:
        print(X_test[sample_id, feature[node_id]])
        print(sample_id)
        print(feature[node_id])
        print(X_test)
        threshold_sign = ">"

    print("decision id node %s : (X_test[%s, %s] (= %s) %s %s)"
          % (node_id,
             sample_id,
             feature[node_id],
             X_test[sample_id, feature[node_id]],
             threshold_sign,
             threshold[node_id]))

ValueError: Number of features of the model must match the input. Model n_features is 10 and input n_features is 4 